In [16]:
import arcpy
import psycopg2
import rasterio
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import arcpy
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Get raster data from postgis

In [ ]:
import psycopg2
import rasterio
from rasterio.io import MemoryFile
import numpy as np
from rasterio.features import bounds as feature_bounds

# Set up your database connection parameters
db_host = "35.223.186.20"
db_port = "5432"
db_name = "postgres"
db_user = "postgres"
db_password = "139571wang"

# Connect to the PostgreSQL database
conn_string = f"dbname='{db_name}' user='{db_user}' host='{db_host}' password='{db_password}' port='{db_port}'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Define the raster_table and column name
raster_table = "raster_table"
raster_column = "rast"

# Fetch the raw raster data from the PostGIS database
cursor.execute(f"SELECT ST_AsBinary({raster_column}) FROM {raster_table};")
raster_data = cursor.fetchone()

if raster_data:
    # Read the binary raster data using rasterio
    with MemoryFile(raster_data[0]) as memfile:
        with memfile.open() as dataset:
            # Get raster data and metadata
            profile = dataset.profile
            data = dataset.read()

            # Define the output path and save the raster data to a file
            raster_output_path = r"C:/test/DEM_raster.tif"
            with rasterio.open(raster_output_path, "w", **profile) as dest:
                dest.write(data)

else:
    print("No raster data found in the table.")

# Close the database connection
cursor.close()
conn.close()


# smple 0.00365% of the dataset

In [9]:
arcpy.management.Resample(r"C:\Users\Maochuan\OneDrive\桌面\lab3\DEM.tif", r"C:\Users\Maochuan\OneDrive\桌面\lab3\resampled_dem.tif", "5000 5000", "NEAREST")

# Convert raster into point shapefile

In [ ]:
arcpy.conversion.RasterToPoint("resampled_dem.tif", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\RasterT_resampl1", "Value")

# IDW interpolation

In [ ]:
out_raster = arcpy.sa.Idw("RasterT_DEM_Res1", "grid_code", 500, 2, "VARIABLE 12", None); out_raster.save(r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\Idw_RasterT_2")

# Kriging 

In [ ]:
out_surface_raster = arcpy.sa.Kriging("RasterT_DEM_Res2", "grid_code", "Spherical # # # #", 2220, "VARIABLE 100", None); out_surface_raster.save(r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\Kriging_Rast2")

# RBF


In [ ]:
arcpy.ga.RadialBasisFunctions("RasterT_DEM_Res2", "grid_code", None, r"C:\Users\Maochuan\OneDrive\桌面\lab3\RBF_DEM.tif", 2220, "NBRTYPE=Standard S_MAJOR=213676.888081046 S_MINOR=213676.888081046 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)

In [ ]:
arcpy.ga.CrossValidation("idw_table.lyr", r"C:\test\IDW_ele_CV.shp")
arcpy.ga.CrossValidation("krig_table.lyr", r"C:\test\Kriging_ele_CV.shp")
arcpy.ga.CrossValidation("rbf_table.lyr", r"C:\test\rbf_ele_CV.shp")

In [27]:
# Convert attribute table to a NumPy array and pd DataFrame
idw_cv_array = arcpy.da.FeatureClassToNumPyArray(r"C:\test\IDW_ele_CV.shp", ["FID", "Error"])
idw_cv_pd = pd.DataFrame(idw_cv_array, columns=["FID", "Error"])

Kriging_cv_array = arcpy.da.FeatureClassToNumPyArray(r"C:\test\Kriging_ele_cv.shp", ["FID", "Error"])
Kriging_cv_pd = pd.DataFrame(Kriging_cv_array, columns=["FID", "Error"])

RBF_cv_array = arcpy.da.FeatureClassToNumPyArray(r"C:\test\RBF_ele_cv.shp", ["FID", "Error"])
RBF_cv_pd = pd.DataFrame(RBF_cv_array, columns=["FID", "Error"])

# Calculate squared error
idw_cv_pd['squared_error'] = idw_cv_pd['Error'] ** 2
Kriging_cv_pd['squared_error'] = Kriging_cv_pd['Error'] ** 2
RBF_cv_pd['squared_error'] = RBF_cv_pd['Error'] ** 2

# Calculate RMSE
idw_rmse_ele = numpy.sqrt(idw_cv_pd['squared_error'].mean())
Kriging_rmse_ele = numpy.sqrt(Kriging_cv_pd['squared_error'].mean())
RBF_rmse_ele = numpy.sqrt(RBF_cv_pd['squared_error'].mean())

#Print and compare results
print(f'The RMSE for IDW is: {idw_rmse}')
print(f'The RMSE for Kriging is: {Kriging_rmse}')
print(f'The RMSE for RBF is: {RBF_rmse}')

The RMSE for IDW is: 12.495551541225144
The RMSE for Kriging is: 8.8461041415266
The RMSE for RBF is: 9.600473079176721


In [ ]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.223.186.20',
                             database = 'postgres',
                             user = 'postgres',
                             password = '139571wang')

# Set the name of the new table in the PostgreSQL database
output_table = "elevation_assesment_table"

# Use psycopg2 to create the table in the PostgreSQL database
cursor = connection.cursor()
cursor.execute(f"CREATE TABLE {output_table} (method text, rmse double precision)")

# Use psycopg2 to copy the data from the Geodatabase table to the PostgreSQL table
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('IDW',idw_rmse_ele))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('Kriging',Kriging_rmse_ele))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('RBF',RBF_rmse_ele))


# Commit the changes to the PostgreSQL database
connection.commit()

# Close the PostgreSQL connection and cursor
cursor.close()
connection.close()

# Interpolation assessment


In [1]:
import arcpy
import os
import numpy as np
from arcpy.sa import *

# 1. Convert the true DEM and the interpolated rasters to point datasets
arcpy.conversion.RasterToPoint("resampled_dem1.tif", "true_dem_points.shp")
arcpy.conversion.RasterToPoint("kriging.tif", "kriging_points.shp")
arcpy.conversion.RasterToPoint("idw.tif", "idw_points.shp")
arcpy.conversion.RasterToPoint("rbf.tif", "rbf_points.shp")

# 2. Sample the true DEM point dataset at the same locations as the interpolated point datasets
arcpy.sa.ExtractValuesToPoints("kriging_points.shp", "resampled_dem1.tif", "kriging_points_with_true_elevation.shp")
arcpy.sa.ExtractValuesToPoints("idw_points.shp", "resampled_dem1.tif", "idw_points_with_true_elevation.shp")
arcpy.sa.ExtractValuesToPoints("rbf_points.shp", "resampled_dem1.tif", "rbf_points_with_true_elevation.shp")

# Function to calculate error metrics
def calculate_errors(true_elevations, interpolated_elevations):
    n = len(true_elevations)
    mae = sum(abs(true - pred) for true, pred in zip(true_elevations, interpolated_elevations)) / n
    mse = sum((true - pred) ** 2 for true, pred in zip(true_elevations, interpolated_elevations)) / n
    rmse = np.sqrt(mse)
    return mae, mse, rmse

# 3. Calculate error metrics (MAE, MSE, RMSE) for each interpolated point dataset
datasets = ["kriging", "idw", "rbf"]

for dataset in datasets:
    input_shapefile = f"{dataset}_points_with_true_elevation.shp"
    true_elevations = []
    interpolated_elevations = []

    with arcpy.da.SearchCursor(input_shapefile, ["RASTERVALU", "GRID_CODE"]) as cursor:
        for row in cursor:
            true_elevations.append(row[0])
            interpolated_elevations.append(row[1])

    mae, mse, rmse = calculate_errors(true_elevations, interpolated_elevations)
    print(f"{dataset}: MAE = {mae}, MSE = {mse}, RMSE = {rmse}")


# Insert the error metrics for each interpolation method
for dataset in datasets:
    # 5. Create a point layer showing the differences between the ground truth and model predictions
    output_shapefile = f"{dataset}_differences.shp"
    arcpy.management.Copy(input_shapefile, output_shapefile)

    arcpy.management.AddField(output_shapefile, "difference", "DOUBLE")
    with arcpy.da.UpdateCursor(output_shapefile, ["RASTERVALU", "GRID_CODE", "difference"]) as cursor:
        for row in cursor:
            true_elevation = row[0]
            interpolated_elevation = row[1]
            difference = true_elevation - interpolated_elevation
            row[2] = difference
            cursor.updateRow(row)


kriging: MAE = 4452.514209683938, MSE = 49080492.96334203, RMSE = 7005.747138124672
idw: MAE = 4464.159534874823, MSE = 49468383.16601199, RMSE = 7033.376370279923
rbf: MAE = 4451.475144462773, MSE = 49198854.4187933, RMSE = 7014.189505480537


# Clip to the extent of Minnesota

In [ ]:
arcpy.analysis.Clip("kriging_difference", r"BOUNDARIES OF MINNESOTA\STATE", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\kriging_difference_Clip", None)

In [ ]:
arcpy.analysis.Clip("idw_differences", r"BOUNDARIES OF MINNESOTA\STATE", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\idw_difference_Clip", None)

In [ ]:
arcpy.analysis.Clip("rbf_differences", r"BOUNDARIES OF MINNESOTA\STATE", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\rbf_difference_Clip", None)

# upload kriging to postgis database


In [1]:
arcpy.ddd.Int("rbf.tif", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\Int_kriging1")

<Result 'C:\\Users\\Maochuan\\OneDrive\\文档\\ArcGIS\\Projects\\arc2_lab3\\arc2_lab3.gdb\\Int_rbf1'>

In [2]:
arcpy.conversion.RasterToPolygon(r"New Group Layer\Int_kriging1", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\RasterT_Int_kri2", "SIMPLIFY", "Value", "MULTIPLE_OUTER_PART", None)

<Result 'C:\\Users\\Maochuan\\OneDrive\\文档\\ArcGIS\\Projects\\arc2_lab3\\arc2_lab3.gdb\\RasterT_Int_rbf1'>

# REPROJECT  

In [30]:
arcpy.management.Project("RasterT_Int_kri2", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\RasterT_Int_kri2_Project", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "WGS_1984_(ITRF00)_To_NAD_1983", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'C:\\Users\\Maochuan\\OneDrive\\文档\\ArcGIS\\Projects\\arc2_lab3\\arc2_lab3.gdb\\rbf_poly_Project'>

# upload using shp2pgsql

In [1]:
import subprocess
import os

# Set the necessary parameters
shp2pgsql_path = r"C:\Program Files\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"C:\Program Files\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"C:\test\kriging_elevation_finall.shp"
srid = 4326
schema_table = "public.kriging_elevationn"
host = "35.223.186.20"
port = 5432
user = "postgres"
db_name = "postgres"
password = "139571wang"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table}"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")


Return code: 0
Error message: could not print result table: Invalid argument
Field id is an FTDouble with width 10 and precision 0
Field gridcode is an FTDouble with width 10 and precision 0
Field shape_leng is an FTDouble with width 19 and precision 11
Field shape_le_1 is an FTDouble with width 19 and precision 11
Field shape_area is an FTDouble with width 19 and precision 11
Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]



In [ ]:
import psycopg2
from flask import Flask, jsonify
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

@app.route('/kriging_elevation', methods=['GET'])
def get_polygon():
    conn = psycopg2.connect(host='35.223.186.20',
                            database='postgres',
                            user='postgres',
                            password='139571wang')

    cur = conn.cursor(cursor_factory=RealDictCursor)
    cur.execute(sql.("SELECT gridcode, ST_AsGeoJSON(geom)::json AS geometry FROM kriging_elevation"))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)


# Download geojson from url using flask

In [3]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.27.164.176:5000/kriging_elevation"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"C:\Users\Maochuan\OneDrive\桌面\lab3"
    output_file_path = os.path.join(output_folder, "kriging_elevation.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")


Output file path: C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_elevation.geojson
GeoJSON file saved as C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_elevation.geojson


# Convert geojson into esri geojson

In [4]:
import json

# Read input JSON data from file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_elevation.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": polygon_data["geometry"]["coordinates"]
            },
            "properties": {
                "elevation": polygon_data["gridcode"]
            }
        }
        for polygon_data in input_data
    ]
}

# Write output JSON data to file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_elevation_final.json', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)


# Json to feature class

In [5]:
arcpy.conversion.JSONToFeatures(r"C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_elevation_final.json", r"C:\test\kriging_elevation_final.shp", "POLYGON")

<Result 'C:\\test\\kriging_elevation_final.shp'>

# upload feature class to arconline

In [ ]:
import os
import zipfile
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Function to zip shapefile
def zip_shapefile(shapefile_path):
    with zipfile.ZipFile(shapefile_path + '.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_name in os.listdir(os.path.dirname(shapefile_path)):
            if file_name.startswith(os.path.basename(shapefile_path).split('.')[0]) and not file_name.endswith('.zip'):
                zf.write(os.path.join(os.path.dirname(shapefile_path), file_name), arcname=file_name)

# Connect to your ArcGIS Online organization
gis = GIS("home")

# Set the input shapefile path
input_shapefile = r'C:\test\kriging_elevation_final.shp'

# Zip the shapefile
zip_shapefile(input_shapefile)

# Upload the zipped shapefile as a new item
item = gis.content.add({'type': 'Shapefile'}, input_shapefile + '.zip')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)


# upload kriging_difference point accuracy assesment layer to postgis database


reproject

In [2]:
arcpy.management.Project("kriging_difference_Clip", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\kriging_difference_C_Project", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "WGS_1984_(ITRF00)_To_NAD_1983", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'C:\\Users\\Maochuan\\OneDrive\\文档\\ArcGIS\\Projects\\arc2_lab3\\arc2_lab3.gdb\\kriging_difference_C_Project'>

In [3]:
import subprocess
import os

# Set the necessary parameters
shp2pgsql_path = r"C:\Program Files\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"C:\Program Files\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"C:\test\Kriging_difference_reprojected.shp"
srid = 4326
schema_table = "public.Kriging_Difference"
host = "35.223.186.20"
port = 5432
user = "postgres"
db_name = "postgres"
password = "139571wang"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table}"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")


Return code: 0
Error message: could not print result table: Invalid argument
Field pointid is an FTDouble with width 10 and precision 0
Field grid_code is an FTDouble with width 13 and precision 11
Field rastervalu is an FTDouble with width 10 and precision 0
Field difference is an FTDouble with width 19 and precision 11
Shapefile type: Point
Postgis type: POINT[2]



In [ ]:
import psycopg2
from flask import Flask, jsonify
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

@app.route('/kriging_difference', methods=['GET'])
def get_polygon():
    conn = psycopg2.connect(host='35.223.186.20',
                            database='postgres',
                            user='postgres',
                            password='139571wang')

    cur = conn.cursor(cursor_factory=RealDictCursor)
    # Cast "difference" to double precision in the SQL query
    cur.execute(sql.SQL("SELECT difference::double precision, ST_AsGeoJSON(geom)::json AS geometry FROM kriging_difference"))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)


In [5]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.27.164.176:5000/kriging_difference"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"C:\Users\Maochuan\OneDrive\桌面\lab3"
    output_file_path = os.path.join(output_folder, "kriging_difference.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")


Output file path: C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_difference.geojson
GeoJSON file saved as C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_difference.geojson


In [10]:
import json

# Read input JSON data from file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_difference.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": point_data["geometry"]["coordinates"]
            },
            "properties": {
                "difference": point_data["difference"]
            }
        }
        for point_data in input_data
    ]
}

# Write output JSON data to file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_difference_esri.geojson', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)


In [11]:
arcpy.conversion.JSONToFeatures(r"C:\Users\Maochuan\OneDrive\桌面\lab3\kriging_difference_esri.geojson", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\kriging_differ_JSONToFeature", "POINT")

<Result 'C:\\Users\\Maochuan\\OneDrive\\文档\\ArcGIS\\Projects\\arc2_lab3\\arc2_lab3.gdb\\kriging_differ_JSONToFeature'>

In [ ]:
import os
import zipfile
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Function to zip shapefile
def zip_shapefile(shapefile_path):
    with zipfile.ZipFile(shapefile_path + '.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_name in os.listdir(os.path.dirname(shapefile_path)):
            if file_name.startswith(os.path.basename(shapefile_path).split('.')[0]) and not file_name.endswith('.zip'):
                zf.write(os.path.join(os.path.dirname(shapefile_path), file_name), arcname=file_name)

# Connect to your ArcGIS Online organization
gis = GIS("home")

# Set the input shapefile path
input_shapefile = r'C:\test\Kriging_difference_final.shp'

# Zip the shapefile
zip_shapefile(input_shapefile)

# Upload the zipped shapefile as a new item
item = gis.content.add({'type': 'Shapefile'}, input_shapefile + '.zip')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)


In [ ]:
export FLASK_APP=app.py
flask run --host=0.0.0.0